In [2]:
import os
import torch
from torchvision import transforms
from PIL import Image, ImageEnhance
import random

# Define your dataset paths
dataset_path = 'Dataset'  # Change this to your dataset path
train_path = os.path.join(dataset_path, 'training')
val_path = os.path.join(dataset_path, 'validation')
augmented_path = os.path.join(dataset_path, 'augmented')  # Path for augmented images

# Create a new folder for augmented images
os.makedirs(augmented_path, exist_ok=True)

# Class names (assuming you have 10 classes)
class_names = os.listdir(train_path)  # List of class directories

# Transformations
zoom_in_transform = transforms.RandomAffine(degrees=0, scale=(1.2, 1.2))  # Zoom in
zoom_out_transform = transforms.RandomAffine(degrees=0, scale=(0.8, 0.8))  # Zoom out
rotation_transform = transforms.RandomRotation(degrees=15)  # Rotate
shift_transform = transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Shift
noise_transform = transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.05)  # Gaussian noise

def sharpen_transform(img):
    if isinstance(img, Image.Image):  # Check if the image is PIL format
        return ImageEnhance.Sharpness(img).enhance(1.2)  # Sharpen
    return img

perspective_transform = transforms.RandomPerspective(distortion_scale=0.2, p=1.0)  # Perspective distortion

# Helper functions
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# Function to apply chosen transformations
def apply_transforms(img, transforms_list):
    for transform in transforms_list:
        if isinstance(transform, transforms.Lambda):  # Special handling for noise transforms
            img = to_pil(transform(to_tensor(img)))
        elif callable(transform):  # For the custom sharpen_transform
            img = transform(img)
        else:
            img = transform(img)
    return img

# Function to generate augmented images with random transformations
def generate_augmented_images(img, num_images=4):
    augmented_images = []
    
    available_transforms = [
        zoom_in_transform,
        zoom_out_transform,
        rotation_transform,
        shift_transform,
        noise_transform,
        sharpen_transform,
        perspective_transform
    ]
    
    for _ in range(num_images):
        num_transforms = random.randint(2, 4)  # Choose between 2 to 4 transformations
        chosen_transforms = random.sample(available_transforms, num_transforms)
        
        # Apply the chosen transforms
        augmented_img = apply_transforms(img, chosen_transforms)
        
        # Append the augmented image to the list
        augmented_images.append(augmented_img)
    
    return augmented_images

# Process each class
for class_name in class_names:
    class_path = os.path.join(train_path, class_name)
    images = os.listdir(class_path)
    
    # Create a folder for the class in augmented directory
    class_augmented_path = os.path.join(augmented_path, class_name)
    os.makedirs(class_augmented_path, exist_ok=True)
    
    # Load images and generate augmented images
    for image_file in images:
        image_path = os.path.join(class_path, image_file)
        img = Image.open(image_path)
        
        # Generate a number of augmented images to reach 22-25k total per class
        current_count = len(os.listdir(class_augmented_path))
        target_count = 25000  # Target number of augmented images
        
        while current_count < target_count:
            augmented_images = generate_augmented_images(img, num_images=4)
            for i, augmented_img in enumerate(augmented_images):
                save_path = os.path.join(class_augmented_path, f"aug_{current_count + i + 1}.jpg")
                augmented_img.save(save_path)
            current_count += len(augmented_images)

print("Data augmentation completed and images saved.")


NotADirectoryError: [Errno 20] Not a directory: 'Dataset/training/~$training_data.xlsx'

In [2]:
import os
import random
from PIL import Image, ImageEnhance
import torch
from torchvision import transforms

# Define dataset paths
dataset_path = 'Dataset'
train_path = os.path.join(dataset_path, 'training')
augmented_path = os.path.join(dataset_path, 'aug_train')  # Changed to aug_train

# Create a folder for augmented images
os.makedirs(augmented_path, exist_ok=True)

# Get class names
class_names = os.listdir(train_path)

# Transformations
transformations = {
    "horizontal_flip": transforms.RandomHorizontalFlip(p=1.0),
    "vertical_flip": transforms.RandomVerticalFlip(p=1.0),
    "rotation": transforms.RandomRotation(degrees=20),  # ±20 degrees rotation
    "zoom_in": transforms.RandomAffine(degrees=0, scale=(1.15, 1.15)),  # Zoom in (±15%)
    "zoom_out": transforms.RandomAffine(degrees=0, scale=(0.85, 0.85)),  # Zoom out (±15%)
    "elastic_transform": transforms.RandomAffine(degrees=0, shear=(10, 10)),  # Simulated elastic distortion with shear ±10°
    "gaussian_blur": transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 1.0)),
    "random_crop": transforms.RandomResizedCrop(size=(224, 224), scale=(0.85, 1.0)),  # Crop between 85% and 100%
    "shear": transforms.RandomAffine(degrees=0, shear=10)  # Shear ±10°
}

# Helper functions
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# Custom function for Gaussian noise
def gaussian_noise(img):
    noise = torch.randn_like(to_tensor(img)) * 0.05  # Subtle noise (0.05 std dev)
    img_tensor = to_tensor(img)
    return to_pil(img_tensor + noise)  # Convert back to PIL format

# Function to apply selected transformations
def apply_transforms(img, transform_list):
    applied_transforms = set()  # Keep track of applied transformations
    for t in transform_list:
        if t == "gaussian_noise":
            if "gaussian_noise" not in applied_transforms:  # Avoid reapplying the same transformation
                img = gaussian_noise(img)
                applied_transforms.add("gaussian_noise")
        else:
            if t not in applied_transforms:  # Avoid reapplying the same transformation
                img = transformations[t](img)
                applied_transforms.add(t)
    return img

# Function to generate random augmented images
def generate_augmented_images(img, num_images=10):
    augmented_images = []
    
    # Define available transformations
    available_transforms = list(transformations.keys()) + ["gaussian_noise"]

    for _ in range(num_images):
        # Randomly choose 2 to 4 transformations
        num_transforms = random.randint(2, 4)
        chosen_transforms = random.sample(available_transforms, num_transforms)
        
        # Apply transformations
        augmented_img = apply_transforms(img, chosen_transforms)
        augmented_images.append(augmented_img)
    
    return augmented_images

# Target number of samples for minority classes
TARGET_TRAIN_SIZE = 15000  # Updated to 15k

# Augment minority classes to reach at least 15,000 samples
for class_name in class_names:
    class_path = os.path.join(train_path, class_name)
    images = os.listdir(class_path)
    
    current_train_size = len(images)
    
    # Create augmented directory for this class
    class_augmented_path = os.path.join(augmented_path, class_name)
    os.makedirs(class_augmented_path, exist_ok=True)

    # Copy original images to augmented directory
    for image_file in images:
        image_path = os.path.join(class_path, image_file)
        img = Image.open(image_path)
        save_path = os.path.join(class_augmented_path, image_file)
        img.save(save_path)

    # Only augment if the current size is less than 15,000
    if current_train_size < TARGET_TRAIN_SIZE:
        print(f"Class {class_name} has {current_train_size} samples, augmenting to reach {TARGET_TRAIN_SIZE}.")
        
        while current_train_size < TARGET_TRAIN_SIZE:
            for image_file in images:
                image_path = os.path.join(class_path, image_file)
                img = Image.open(image_path)
                
                # Calculate number of augmented images needed
                num_augments = TARGET_TRAIN_SIZE - current_train_size
                augmented_images = generate_augmented_images(img, num_images=min(num_augments, 10))
                
                # Save augmented images
                for i, augmented_img in enumerate(augmented_images):
                    current_train_size += 1
                    save_path = os.path.join(class_augmented_path, f"aug_{current_train_size}.jpg")
                    augmented_img.save(save_path)
                    
                    if current_train_size >= TARGET_TRAIN_SIZE:
                        break

print("Data augmentation completed and images saved.")


Class Polyp has 1162 samples, augmenting to reach 15000.
Class Bleeding has 834 samples, augmenting to reach 15000.
Class Foreign Body has 792 samples, augmenting to reach 15000.
Class Erosion has 2694 samples, augmenting to reach 15000.
Class Ulcer has 663 samples, augmenting to reach 15000.
Class Erythema has 691 samples, augmenting to reach 15000.
Class Worms has 158 samples, augmenting to reach 15000.
Class Lymphangiectasia has 796 samples, augmenting to reach 15000.
Class Angioectasia has 1154 samples, augmenting to reach 15000.
Data augmentation completed and images saved.


In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Set data directories
train_data_dir = 'Dataset/augmented_training'
val_data_dir = 'Dataset/validation'

# ImageDataGenerator for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='input'
)

validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='input'
)

# Set steps per epoch to avoid running out of data
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size

# Define the encoder-decoder model
input_img = layers.Input(shape=(224, 224, 3))

# Encoder (Feature extraction)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder (Reconstruction)
x = layers.Conv2DTranspose(512, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

decoded = layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

# Compile the model
autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Show the model architecture
autoencoder.summary()

# Train the model
history = autoencoder.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
autoencoder.summary()

Found 96163 images belonging to 10 classes.
Found 16132 images belonging to 10 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 14, 14, 512)    │     2,359,808 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 28, 28, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 256)    │     1,179,904 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_4 (UpSampling2D)  │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 56, 56, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_5 (UpSampling2D)  │ (None, 112, 112, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 112, 112, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 224, 224, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 224, 224, 3)    │         1,731 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,461,251 (20.83 MB)

 Trainable params: 5,461,251 (20.83 MB)

...

/home/ws-008/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5833s 2s/step - loss: 0.0065 - val_loss: 0.0013
Epoch 2/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 63us/step - loss: 0.0027 - val_loss: 0.0031
Epoch 3/50


/home/ws-008/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5839s 2s/step - loss: 0.0022 - val_loss: 0.0011
Epoch 4/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 50us/step - loss: 0.0017 - val_loss: 6.2602e-04
Epoch 5/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5871s 2s/step - loss: 0.0020 - val_loss: 9.5871e-04
Epoch 6/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 42us/step - loss: 0.0022 - val_loss: 6.0693e-04
Epoch 7/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5701s 2s/step - loss: 0.0018 - val_loss: 8.6676e-04
Epoch 8/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 43us/step - loss: 0.0020 - val_loss: 4.6400e-04
Epoch 9/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5718s 2s/step - loss: 0.0017 - val_loss: 8.9654e-04
Epoch 10/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 46us/step - loss: 0.0023 - val_loss: 0.0019
Epoch 11/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 5730s 2s/step - loss: 0.0016 - val_loss: 8.0911e-04
Epoch 12/50
3005/3005 ━━━━━━━━━━━━━━━━━━━━ 2s 40us/step - loss: 0.0019 - val_loss: 6.1024e-04
Epoch 13/50
1329/3005 ━━━━━━━━━━━━━━━━━━━━ 50:38 2s/step - loss: 0.0016

KeyboardInterrupt: 